# GAbDT

In [14]:
def criar_individuo(atributos):
    """
    A ideia é que possamos explorar melhor o espaço de possibilidades e as possiveis divisões do nosso conjunto de dados e não apenas utilizando a média entre dois atributos.
    """
    atributo = random.choice(atributos)
    operador = random.choice(["<", ">", "<=", ">="])
    threshold = random.uniform(min(df[atributo]), max(df[atributo]))
    return {"premisse": (atributo, operador, threshold)}

coluna_classe="classes"
atributos = df.columns[df.columns != "classes"].tolist()

In [9]:
def aplicar_premissa(df, premissa):
    """
    Esta função aplica as premissas geradas na população inicial dividindo o df em dois novos grupos, df_true e df_false para os que aténdem à premissa e os que não atendem respectivamente
    desta forma podemos posteriormente calcular o erro majoritário gerado pela divisão de cada uma das premissas e encontrar a que melhor divide os dados, essas premissas ainda 
    serão evoluidas ao longo de gerações pelo AG tentando maximizar seu fitness.
    """
    
    atributo, operador, threshold = premissa["premisse"]
    
    if operador == "<":
        df_true = df[df[atributo] < threshold]
        df_false = df[df[atributo] >= threshold]
    elif operador == "<=":
        df_true = df[df[atributo] <= threshold]
        df_false = df[df[atributo] > threshold]
    elif operador == ">":
        df_true = df[df[atributo] > threshold]
        df_false = df[df[atributo] <= threshold]
    elif operador == ">=":
        df_true = df[df[atributo] >= threshold]
        df_false = df[df[atributo] < threshold]
    else:
        raise ValueError(f"Operador desconhecido: {operador}")
    
    return df_true, df_false

In [12]:
def erro_majoritario(df, coluna_classe="classes"):
    """Calcula o erro de classificação por maioria (quanto menor, melhor)."""
    if df.empty:
        return 1  
    classe_majoritaria = df[coluna_classe].value_counts().idxmax()
    erro = sum(df[coluna_classe] != classe_majoritaria)/len(df)
    return erro

In [1]:
def evaluate(df, populacao, coluna_classe="classes"):
    """
    Esta função utiliza a aplicar_premissa para gerar os df_verdadeiro e df_falso e calcula o erro majoritário em ambos, ponderado pelo tamanho dos conjuntos gerados em relação ao df original, ou seja, verificando
    se a premissa separa os dados em conjuntos representativos.
    """
    resultados = []
    for individuo in populacao:
        df_verdadeiro, df_falso = aplicar_premissa(df, individuo)
        
        #Calcula o erro majoritário em cada subconjunto
        erro_true = erro_majoritario(df_verdadeiro, coluna_classe) 
        erro_false = erro_majoritario(df_falso, coluna_classe) 
        
        #Peso proporcional ao tamanho dos conjuntos
        peso_true = len(df_verdadeiro)/len(df) if len(df) > 0 else 0
        peso_false = len(df_falso)/len(df) if len(df) > 0 else 0

        #Fitness é o erro ponderado (quanto menor, melhor)
        fitness = peso_true * erro_true + peso_false * erro_false
        
        resultados.append((individuo, fitness))
    
    return resultados

In [25]:
def torneio(pop_avaliada, k=3):
    """
    Seleção por torneio, seleciona o melhor indivíduo entre k indivíduos aleatórios para ser um dos pais que gera um dos indivíduos da proxima geração, isto é feito duas
    vezes para cada novo indivíduo gerado, selecionando o pai1 e o pai2.
    """
    competidores = random.sample(pop_avaliada, k)
    vencedor = min(competidores, key=lambda x: x[1])  #Menor fitness
    return vencedor[0]  

In [26]:
def crossover_rnd(pai1, pai2):
    #Escolhe cada componente da premissa de um pai ou outro
    atributo = pai1["premisse"][0] if random.random() < 0.5 else pai2["premisse"][0]
    operador = pai1["premisse"][1] if random.random() < 0.5 else pai2["premisse"][1]
    threshold = pai1["premisse"][2] if random.random() < 0.5 else pai2["premisse"][2]
    
    premissa_filho = (atributo, operador, threshold)
    
    return {"premisse": premissa_filho}

In [27]:
def mutacao(individuo, atributos, intervalo_atributos, prob_mut_attr=0.1, prob_mut_op=0.1, prob_mut_val=0.2):
    #Acessa diretamente o dicionário 'premisse' dentro do individuo
    premissa = individuo['premisse']  
    nova_premissa = list(premissa)  

    #Mutação do atributo
    if random.random() < prob_mut_attr:
        nova_premissa[0] = random.choice(atributos)
        min_val, max_val = intervalo_atributos[nova_premissa[0]]
        nova_premissa[2] = random.uniform(min_val, max_val)

    #Mutação do operador
    if random.random() < prob_mut_op:
        operadores = ['<', '<=', '>', '>=']
        nova_premissa[1] = random.choice(operadores)

    #Mutação do valor
    if random.random() < prob_mut_val:
        attr = nova_premissa[0]
        min_val, max_val = intervalo_atributos[attr]
        nova_premissa[2] = random.uniform(min_val, max_val)

    return {'premisse': tuple(nova_premissa)}  # Retorna um dicionário novamente

In [28]:
def calcular_intervalo_atributos(df, atributos):
    """
    Calcula o intervalo de valores para cada atributo numérico do DataFrame.
    """
    intervalo = {}
    for atributo in atributos:
        min_val = df[atributo].min()
        max_val = df[atributo].max()
        intervalo[atributo] = (min_val, max_val)
    return intervalo

In [29]:
def nova_geracao(
    pop_avaliada, 
    atributos, 
    intervalo_atributos, 
    tamanho=100, 
    prob_mut_attr=0.1, 
    prob_mut_op=0.1, 
    prob_mut_val=0.2
):
    """
    Gera uma nova geração com elitismo e operadores genéticos.
    - Mantém o melhor indivíduo.
    - Usa torneio + crossover + mutação para gerar o restante.
    """
    #Elitismo
    elite = min(pop_avaliada, key=lambda x: x[1])[0]  #Pega o melhor (menor fitness)
    
    nova_populacao = [elite]  #Inicia com o elite
    
    while len(nova_populacao) < tamanho:
        #Seleção por torneio
        pai1 = torneio(pop_avaliada)
        pai2 = torneio(pop_avaliada)
        
        #Crossover
        filho = crossover_rnd(pai1, pai2)
        #print('\nFilho', filho)
        
        #Mutação
        filho_mutado = mutacao(
            filho,
            atributos,
            intervalo_atributos,
            prob_mut_attr=prob_mut_attr,
            prob_mut_op=prob_mut_op,
            prob_mut_val=prob_mut_val
        )
        
        nova_populacao.append(filho_mutado)
    
    return nova_populacao

In [30]:
def evoluir_solucoes(df, 
                      atributos, 
                      intervalo_atributos, 
                      profundidade=0, 
                      max_profundidade=5, 
                      n_geracoes=10, 
                      tamanho_pop=200, 
                      taxa_mutacao=0.2, 
                      coluna_classe="classes",
                      min_samples=2):
    """
    Evolui soluções para dividir os dados em cada nó da árvore de decisão com o algoritmo genético.
    """
    #Inicia a população
    populacao = [criar_individuo(atributos) for _ in range(tamanho_pop)]
    
    #Critério de parada
    if profundidade >= max_profundidade or len(df) < min_samples:
        return {
            "folha": True,
            "classes": df[coluna_classe].value_counts().to_dict()
        }

    #Avaliação ao longo das gerações
    for gen in range(n_geracoes):
        avaliacoes = evaluate(df, populacao, coluna_classe)
        melhor_ind, melhor_fit = min(avaliacoes, key=lambda x: x[1])
        populacao = nova_geracao(
            avaliacoes, 
            atributos, 
            intervalo_atributos, 
            tamanho=tamanho_pop, 
            prob_mut_attr=0.1, 
            prob_mut_op=0.1, 
            prob_mut_val=0.2
        )

    #Seleciona a melhor premissa e aplica
    melhor_individuo = melhor_ind
    premissa = melhor_individuo 
    df_esquerda, df_direita = aplicar_premissa(df, melhor_individuo)
    
    if len(df_esquerda) < min_samples or len(df_direita) < min_samples:
        return {
            "folha": True,
            "classes": df[coluna_classe].value_counts().to_dict()
        }
    
    #Verifica o erro majoritário para os subconjuntos
    erro_esquerda = erro_majoritario(df_esquerda, coluna_classe)
    erro_direita = erro_majoritario(df_direita, coluna_classe)

    #Evolui ou para cada ramo separadamente
    if erro_esquerda < erro_threshold:
        no_esquerdo = {
            "folha": True,
            "classes": df_esquerda[coluna_classe].value_counts().to_dict()
        }
    else:
        no_esquerdo = evoluir_solucoes(
            df_esquerda, atributos, intervalo_atributos,
            profundidade + 1, max_profundidade, n_geracoes,
            tamanho_pop, taxa_mutacao, coluna_classe, min_samples
        )

    if erro_direita < erro_threshold:
        no_direito = {
            "folha": True,
            "classes": df_direita[coluna_classe].value_counts().to_dict()
        }
    else:
        no_direito = evoluir_solucoes(
            df_direita, atributos, intervalo_atributos,
            profundidade + 1, max_profundidade, n_geracoes,
            tamanho_pop, taxa_mutacao, coluna_classe, min_samples
        )

    return {
        "folha": False,
        "premissa": premissa,
        "contagem_total": df[coluna_classe].value_counts().to_dict(),
        "contagem_esquerda": df_esquerda[coluna_classe].value_counts().to_dict(),
        "contagem_direita": df_direita[coluna_classe].value_counts().to_dict(),
        "esquerda": no_esquerdo,
        "direita": no_direito
    }

In [31]:
import matplotlib.pyplot as plt
import warnings

def plot_tree_custom(node, x=0, y=0, dx=3.0, dy=1.0, ax=None, depth=0, pos_dict=None, parent_pos=None):
    """
    Representação visual da AD.
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(35, 30), dpi=350)
    
        ax.axis("off")
        pos_dict = {"max_y": 0, "min_y": 0, "depth_max": get_tree_depth(node)}
        plot_tree_custom(node, x, y, dx, dy, ax, depth, pos_dict)
        
        plt.savefig("arvore_custom.png", bbox_inches="tight")
        warnings.filterwarnings("ignore", category=UserWarning)
        plt.show()
        return

    #Ajuste dinâmico com base na profundidade total
    spacing_factor = pos_dict["depth_max"] - depth + 1
    current_dx = dx * spacing_factor
    current_dy = dy * spacing_factor * 0.5  
    
    #Folha
    if node.get("folha", False):
        classes = node.get("classes", {})
        total = sum(classes.values())

        text = "Folha\n"
        text += f"Total: {total}\n"
        text += "Classes:\n"
        for cls, count in sorted(classes.items()):
            text += f"  {cls}: {count} ({count/total:.1%})\n"

        ax.text(x, y, text,
                bbox=dict(facecolor="#c1f0c1", alpha=1.0, edgecolor="darkgreen", boxstyle="round,pad=0.5"),
                ha="center", va="center", fontsize=10)

        if parent_pos:
            ax.plot([parent_pos[0], x], [parent_pos[1], y],
                    color="#555555", linestyle="-", linewidth=1.5, alpha=1.0)

        pos_dict["max_y"] = max(pos_dict["max_y"], y)
        pos_dict["min_y"] = min(pos_dict["min_y"], y)
        return y

    #Nó interno
    premissa = node.get("premissa", {}).get("premisse", ("?", "?", "?"))
    total_counts = node.get("contagem_total", {})
    left_counts = node.get("contagem_esquerda", {})
    right_counts = node.get("contagem_direita", {})
    total = sum(total_counts.values())

    text = f"{premissa[0]}\n{premissa[1]} {premissa[2]:.2f}\n"
    text += f"Total: {total}\n"
    text += "Distribuição:\n"
    for cls, count in sorted(total_counts.items()):
        text += f"  {cls}: {count} ({count/total:.1%})\n"

    ax.text(x, y, text,
            bbox=dict(facecolor="#b0e0e6", alpha=1.0, edgecolor="darkblue", boxstyle="round,pad=0.5"),
            ha="center", va="center", fontsize=10)

    if parent_pos:
        ax.plot([parent_pos[0], x], [parent_pos[1], y],
                color="#555555", linestyle="-", linewidth=1.5, alpha=1.0)

    #Coordenadas dos filhos
    left_y = y - current_dy
    right_y = y + current_dy

    left_child_y = plot_tree_custom(node["esquerda"], x + current_dx, left_y,
                                    dx, dy, ax, depth + 1, pos_dict, parent_pos=(x, y))
    left_text = "True\n"
    ax.text(x + current_dx / 3, (y + left_child_y) / 2, left_text,
            ha="right", va="center", fontsize=15, color="#1f77b4",
           )

    right_child_y = plot_tree_custom(node["direita"], x + current_dx, right_y,
                                     dx, dy, ax, depth + 1, pos_dict, parent_pos=(x, y))
    right_text = "False\n"
    ax.text(x + current_dx / 3, (y + right_child_y) / 2, right_text,
            ha="left", va="center", fontsize=15, color="#d62728",
            )

    return y


def get_tree_depth(node):
    """Calcula a profundidade máxima da árvore."""
    if node.get("folha", False):
        return 0
    return 1 + max(get_tree_depth(node["esquerda"]), get_tree_depth(node["direita"]))

In [32]:
def predict_custom_tree(tree, X):
    """
    Faz predições usando o formato de nossa árvore de decisão baseada em dicionários com estrutura customizada.
    """
    def predizer_linha(no, linha):
        if no["folha"]:
            #Retorna a classe com maior frequência
            return max(no["classes"].items(), key=lambda x: x[1])[0]
        
        atributo, operador, valor = no["premissa"]["premisse"]
        entrada = linha[atributo]

        #Aplica operadores
        if operador == "<":
            if entrada < valor:
                return predizer_linha(no["esquerda"], linha)
            else:
                return predizer_linha(no["direita"], linha)
        elif operador == "<=":
            if entrada <= valor:
                return predizer_linha(no["esquerda"], linha)
            else:
                return predizer_linha(no["direita"], linha)
        elif operador == ">":
            if entrada > valor:
                return predizer_linha(no["esquerda"], linha)
            else:
                return predizer_linha(no["direita"], linha)
        elif operador == ">=":
            if entrada >= valor:
                return predizer_linha(no["esquerda"], linha)
            else:
                return predizer_linha(no["direita"], linha)
        else:
            raise ValueError(f"Operador desconhecido: {operador}")
    
    #Aplica para todas as linhas
    return [predizer_linha(tree, linha) for _, linha in X.iterrows()]